## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2023xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [2]:
from datetime import datetime
import json
from tab_dataset.cdataset import Cdataset
import pandas as pd
import ntv_pandas

def analyse_integrite(data, fields, affiche=True, indic=True):
    '''analyse les relations du DataFrame 'data' définies dans le dict 'fields'.
    Le nombre de ligne en erreur par relation (dict) est retourné et optionnellement ('indic') affiché. 
    La liste des lignes en erreur est optionnellement ajoutée ('affiche') à 'data' sous forme de champs booléens par relation.
    '''
    dic_errors = Cdataset(data).check_relationship(fields)
    dic_count = {name: len(errors) for name, errors in dic_errors.items()}
    if affiche:
        for name, total in dic_count.items():
            print('{:<50} {:>5}'.format(name, total))
    if indic:
        data['ok'] = True
        for name, errors in dic_errors.items():
            data[name] = True
            data.loc[errors, name] = False
            data['ok'] = data['ok'] & data[name] 
    return dic_count

def add_nbre_pdc(data, affiche=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    if affiche:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [3]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08',
         '2023-09-06', '2023-09-19', '2023-10-04', '2023-10-21', '2023-10-31', '2023-11-23', '2023-12-14']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
#    'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
#log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
    'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
    #'chemin': 'C:\\Users\\phili\\github\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'
      }
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  72902


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
fields = [
 # relation unicité des pdl
     { "name": "index",
       "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 0,4 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  2807
nombre de pdc en itinerance   :  70095


In [6]:
# arborescence des champs et nombre de valeurs différentes
print(Cdataset(itinerance_init).tree())

-1: root-derived (70095)
   0 : index (0 - 70095)
   1 : contact_operateur (69950 - 145)
   2 : nom_enseigne (66510 - 3585)
   3 : coordonneesXY (49199 - 20896)
   4 : adresse_station (49595 - 20500)
   5 : id_station_itineranc (38239 - 31856)
   6 : nom_station (48934 - 21161)
   7 : implantation_station (70090 - 5)
   8 : nbre_pdc (70041 - 54)
   9 : condition_acces (70093 - 2)
   10: horaires (69585 - 510)
   11: station_deux_roues (70089 - 6)
   12: id_pdc_itinerance (2104 - 67991)
   13: date_maj (58574 - 11521)
   14: last_modified (69604 - 491)


-----------------------
## 4 - Bilan initial intégrité
- 12 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                           4206
contact_operateur - id_station_itinerance            641
nom_enseigne - id_station_itinerance                1254
coordonneesXY - id_station_itinerance               3230
id_station_itinerance - id_pdc_itinerance            924
nom_station - id_station_itinerance                  954
implantation_station - id_station_itinerance         407
nbre_pdc - id_station_itinerance                    2517
condition_acces - id_station_itinerance               27
horaires - id_station_itinerance                      33
station_deux_roues - id_station_itinerance          2169
adresse_station - coordonneesXY                     2877

nombre d'enregistrements sans erreurs :  61578
nombre d'enregistrements avec au moins une erreur :  8517
taux d'erreur :  12  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- 25% des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de 50% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'], ' soit ', round(log['sans_doublons_pdc']/log['init_ko'] * 100), ' %')

nombre de doublons pdc :  2104  soit  25  %
nombre de pdc sans doublon   :  6413  soit  75  %


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             45
nom_enseigne - id_station_itinerance                  48
coordonneesXY - id_station_itinerance                998
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  635
implantation_station - id_station_itinerance         355
nbre_pdc - id_station_itinerance                     976
condition_acces - id_station_itinerance               12
horaires - id_station_itinerance                      18
station_deux_roues - id_station_itinerance           479
adresse_station - coordonneesXY                     2324

nombre d'enregistrements sans erreurs :  2076
nombre d'enregistrements avec au moins une erreur :  4337


-----------------------
## 6 - Séparation doublons station - date de maj
- la suppression des anciens pdc permet de réduire de 25% supplémentaire le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 35 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  1034  soit  12  %
nombre de pdc sans doublon   :  3303


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                   5
coordonneesXY - id_station_itinerance                323
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  210
implantation_station - id_station_itinerance         107
nbre_pdc - id_station_itinerance                     583
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                      15
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                     1807

nombre d'enregistrements sans erreurs :  135
nombre d'enregistrements avec au moins une erreur :  3168  soit  73  %


In [12]:
# structure des données présentant une erreur et nombre de valeurs
print(Cdataset(itinerance_5).tree())

-1: root-derived (3168)
   0 : index (0 - 3168)
   1 : contact_operateur (3127 - 41)
   2 : nom_enseigne (3015 - 153)
      9 : condition_acces (151 - 2)
   3 : coordonneesXY (2412 - 756)
   4 : adresse_station (1962 - 1206)
   5 : id_station_itineranc (1324 - 1844)
      13: date_maj (1394 - 450)
   6 : nom_station (1959 - 1209)
   7 : implantation_station (3163 - 5)
   8 : nbre_pdc (3149 - 19)
   10: horaires (3136 - 32)
   11: station_deux_roues (3163 - 5)
   12: id_pdc_itinerance (0 - 3168)
   14: last_modified (3098 - 70)


In [13]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  3  erreurs :  109


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [14]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  3168
total des doublons à supprimer :  3138
nombre de pdc avec controles ok :  63789


In [15]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [16]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, affiche=False)
log['bilan_erreurs'] = sum(res.values())
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [17]:
# structure des données bonnes
print(Cdataset(itinerance_ok.loc[:, mandatory]).tree())

-1: root-derived (63789)
   4 : id_station_itineranc (33852 - 29937)
      1 : nom_enseigne (26477 - 3460)
      2 : coordonneesXY (10173 - 19764)
         0 : contact_operateur (19636 - 128)
         3 : adresse_station (748 - 19016)
      5 : nom_station (10202 - 19735)
      6 : implantation_station (29932 - 5)
      7 : nbre_pdc (29883 - 54)
      8 : condition_acces (29935 - 2)
      9 : horaires (29450 - 487)
      10: station_deux_roues (29931 - 6)
      19: paiement_autre (29930 - 7)
      22: restriction_gabarit (29848 - 89)
      24: last_modified (29509 - 428)
   11: id_pdc_itinerance (0 - 63789)
   12: puissance_nominale (63683 - 106)
   13: prise_type_ef (63781 - 8)
   14: prise_type_2 (63782 - 7)
   15: prise_type_combo_ccs (63782 - 7)
   16: prise_type_chademo (63782 - 7)
   17: prise_type_autre (63783 - 6)
   18: paiement_acte (63781 - 8)
   20: reservation (63782 - 7)
   21: accessibilite_pmr (63785 - 4)
   23: date_maj (53141 - 10648)


### Indicateurs

In [18]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [19]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  72  % des écarts

sav@izivia.com 758
supervision-ev.france@totalenergies.com 504
support@alizecharge.fr 364
assistance-commerciale@e-vadea.fr 327
info@ionity.eu 326


In [20]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, affiche=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko           2366
75  %


In [21]:
# stockage des indicateurs
log |= indic
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (3) (109 pdc)
- exemple station 'FR55CP92140' avec 29 pdc ! : 5 coordonnées XY, 5 nom_station, 2 nbre_pdc)

In [22]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
62,340,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.977306, 45.698458]",BP MANISSIEUX - SAINT-PRIEST,FR55CPBP698,BP MANISSIEUX - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0239151,2022-11-03,2023-12-14T08:19:26.849000+00:00
63,341,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.977306, 45.698458]",BP MANISSIEUX - SAINT-PRIEST,FR55CPBP698,BP MANISSIEUX - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0239152,2022-11-03,2023-12-14T08:19:26.849000+00:00
64,342,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.977306, 45.698458]",BP MANISSIEUX - SAINT-PRIEST,FR55CPBP698,BP MANISSIEUX - SAINT-PRIEST - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE69800GIRVATMX0239153,2022-11-03,2023-12-14T08:19:26.849000+00:00
97,503,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553311,2022-11-03,2023-12-14T08:19:26.849000+00:00
98,504,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553321,2022-11-03,2023-12-14T08:19:26.849000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751,65865,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",23 av. de Corinthe,FRTCBP01835,AMP | 23 av. de Corinthe,Voirie,4,Accès libre,24/7,FALSE,FRTCBE008379,2023-10-17,2023-12-14T08:18:52.971000+00:00
2752,65866,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",23 av. de Corinthe,FRTCBP01835,AMP | 23 av. de Corinthe,Voirie,4,Accès libre,24/7,FALSE,FRTCBE008380,2023-10-17,2023-12-14T08:18:52.971000+00:00
2753,65867,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",23 av. de Corinthe,FRTCBP01835,AMP | 23 av. de Corinthe,Voirie,4,Accès libre,24/7,FALSE,FRTCBE008381,2023-10-17,2023-12-14T08:18:52.971000+00:00
2754,65868,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",23 av. de Corinthe,FRTCBP01835,AMP | 23 av. de Corinthe,Voirie,4,Accès libre,24/7,FALSE,FRTCBE008382,2023-10-17,2023-12-14T08:18:52.971000+00:00


In [45]:
itinerance_4.loc[itinerance_4.id_station_itinerance == 'FR55CP92140', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
97,503,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553311,2022-11-03,2023-12-14T08:19:26.849000+00:00
98,504,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553321,2022-11-03,2023-12-14T08:19:26.849000+00:00
99,505,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553331,2022-11-03,2023-12-14T08:19:26.849000+00:00
100,506,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553341,2022-11-03,2023-12-14T08:19:26.849000+00:00
101,507,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553351,2022-11-03,2023-12-14T08:19:26.849000+00:00
102,508,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553361,2022-11-03,2023-12-14T08:19:26.849000+00:00
103,509,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628411,2022-11-03,2023-12-14T08:19:26.849000+00:00
104,510,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628421,2022-11-03,2023-12-14T08:19:26.849000+00:00
105,511,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628431,2022-11-03,2023-12-14T08:19:26.849000+00:00
106,512,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628441,2022-11-03,2023-12-14T08:19:26.849000+00:00


### Repartition des defauts par operateur
Répartition pour les 5 opérateurs présentant le plus de défauts

In [44]:
print('PdC avec une incohérence : ')
for operateur in oper:
    print('\n', sum(itinerance_4.contact_operateur == operateur))
    print('Répartition des incohérences ', operateur, ' :')
    for cont in controles:
        #print(cont, sum((itinerance_4['contact_operateur'] == 'supervision-ev.france@totalenergies.com') & ~(itinerance_4[cont])))
        print('{:<50} {:>5}'.format(cont, sum((itinerance_4['contact_operateur'] == operateur) & ~(itinerance_4[cont]))))

PdC avec une incohérence : 

 760
Répartition des incohérences  sav@izivia.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance                   0
coordonneesXY - id_station_itinerance                  0
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                    0
implantation_station - id_station_itinerance           0
nbre_pdc - id_station_itinerance                       0
condition_acces - id_station_itinerance                0
horaires - id_station_itinerance                       0
station_deux_roues - id_station_itinerance             0
adresse_station - coordonneesXY                      758

 504
Répartition des incohérences  supervision-ev.france@totalenergies.com  :
index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              0
nom_enseigne - id_station_itinerance     

### Cohérence implantation_station - id_station
- 107 pdc sont liés à une erreur de choix d'implantation (couplé à d'autres erreurs)

In [25]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
42,309,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300611,2022-11-03,2023-12-14T08:19:26.849000+00:00
43,310,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Voirie,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300621,2022-11-03,2023-12-14T08:19:26.849000+00:00
44,311,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300631,2022-11-03,2023-12-14T08:19:26.849000+00:00
45,312,contact@e55c.com,ELECTRIC 55 CHARGING,"[7.130067, 43.641297]",COOLWORK - VILLENEUVE-LOUBET,FR55C06270V,COOLWORK - VILLENEUVE-LOUBET,Parking privé à usage public,4,Accès réservé,24/7,FALSE,FR55CE062704364129771300641,2022-11-03,2023-12-14T08:19:26.849000+00:00
926,26511,support@iecharge.io,Vezin-le-Coquet,"[-1.7547021, 48.1252516]","4001 Rue de la Belle-Epine, 35132 Vezin-le-Coquet",FRIENE35353A,Vezin-le-Coquet,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIENE35353A1,2023-12-13,2023-12-14T08:20:44.132000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2719,60802,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE94,Shell Fenioux Est,Voirie,8,Accès libre,24/7,false,FRSHEE944,2023-12-13,2023-12-14T08:20:44.132000+00:00
2720,60803,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRSHEE951,2023-12-13,2023-12-14T08:20:44.132000+00:00
2721,60804,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRSHEE952,2023-12-13,2023-12-14T08:20:44.132000+00:00
2722,60805,customerservice@shellrecharge.com,Shell Fenioux Est,"[-0.60405, 45.8923]","A10 Aire de Fenioux Est, 17350 Fenioux",FRSHEE95,Shell Fenioux Est,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRSHEE953,2023-12-13,2023-12-14T08:20:44.132000+00:00


### Cohérence nom_station - id_station
- 210 pdc sont associés à une station avec un nom non cohérent

In [47]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
47,317,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643461,2022-11-03,2023-12-14T08:19:26.849000+00:00
48,318,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.764347, 43.958037]",AVENUE DU GENERAL DE GAULE - LES ANGLES,FR55CP30133,AVENUE DU GENERAL DE GAULE - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395803747643471,2022-11-03,2023-12-14T08:19:26.849000+00:00
49,319,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684251,2022-11-03,2023-12-14T08:19:26.849000+00:00
50,320,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.752516, 43.962289]",PARKING PRIADES - LES ANGLES,FR55CP30133,PARKING PRIADES - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334395842547684261,2022-11-03,2023-12-14T08:19:26.849000+00:00
51,321,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.768425, 43.958425]",PARKING ELEPHANT BLEU - LES ANGLES,FR55CP30133,PARKING ELEPHANT BLEU - LES ANGLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CE301334396228947525161,2022-11-03,2023-12-14T08:19:26.849000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2754,65868,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",23 av. de Corinthe,FRTCBP01835,AMP | 23 av. de Corinthe,Voirie,4,Accès libre,24/7,FALSE,FRTCBE008382,2023-10-17,2023-12-14T08:18:52.971000+00:00
2763,65877,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",28 bd. Jeanne d’Arc,FRTCBP01835,AMP | 128 boulevard Jeanne d'Arc,Voirie,2,Accès libre,24/7,FALSE,FRTCBE008393,2023-10-17,2023-12-14T08:18:52.971000+00:00
2764,65878,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",28 bd. Jeanne d’Arc,FRTCBP01836,AMP | 128 boulevard Jeanne d'Arc,Voirie,2,Accès libre,24/7,FALSE,FRTCBE008394,2023-10-17,2023-12-14T08:18:52.971000+00:00
2771,65885,supervision-ev.france@totalenergies.com,La Recharge - Aix Marseille Provence,"[5.115036, 43.398722]",59 rue Niels Bohr,FRTCBP01836,AMP | 59 rue Niels Bohr,Voirie,2,Accès libre,24/7,FALSE,FRTCBE008407,2023-10-17,2023-12-14T08:18:52.971000+00:00


### Coherence adresse - coordonnees
- 1807 pdc ont une adresse non cohérente avec les coordonnées géographiques

Toutes les stations Ionity en erreur (326) ont un id_station_itinerance identique à l'id_pdc_itinerance ainsi qu'un ancien pdc de regroupement par station avec un nom d'enseigne différent (IONITY GMBH), ce qui se traduit par des adresses multiples pour une même coordonnée.

In [46]:
itinerance_4.loc[itinerance_4['contact_operateur'] == 'info@ionity.eu', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
931,26943,info@ionity.eu,IONITY GMBH,"[0.366838, 46.701436]","Aire de Poitou-Charentes, A10, 79180 Vouillé",FRIONE124800,IONITY Poitou Charentes,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1248,2023-03-21,2023-12-14T08:19:38.261000+00:00
932,26944,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2023-12-14T08:19:38.261000+00:00
933,26945,info@ionity.eu,IONITY GMBH,"[1.578674, 48.467261]","Aire De Chartres Bois,A11 - 28300 Gasville-Oiséme",FRIONE126500,IONITY Chartres Bois Paris,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1265,2023-03-21,2023-12-14T08:19:38.261000+00:00
934,26946,info@ionity.eu,IONITY GMBH,"[3.918671, 49.242974]","Aire de Vrigny,A4-51390 Vrigny",FRIONE401700,IONITY Vrigny,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4017,2023-03-21,2023-12-14T08:19:38.261000+00:00
935,26949,info@ionity.eu,IONITY GMBH,"[1.416918, 46.313059]","Aire de Boismandé Ouest, A20 - 87160 Saint-Sul...",FRIONE402000,IONITY Boismandé Ouest,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE4020,2023-03-21,2023-12-14T08:19:38.261000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,27764,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440316,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440316,2023-12-13,2023-12-14T08:20:44.132000+00:00
1253,27765,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440317,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440317,2023-12-13,2023-12-14T08:20:44.132000+00:00
1254,27766,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440318,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440318,2023-12-13,2023-12-14T08:20:44.132000+00:00
1255,27767,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE44035,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE44035,2023-12-13,2023-12-14T08:20:44.132000+00:00


In [49]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[1.580008, 48.468782]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
932,26944,info@ionity.eu,IONITY GMBH,"[1.580008, 48.468782]","Aire de Chartres-Gasville, A11 - 28300 Gasvill...",FRIONE126400,IONITY Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,FALSE,FRIONE1264,2023-03-21,2023-12-14T08:19:38.261000+00:00
979,27085,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126401,Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIOYE126401,2023-12-13,2023-12-14T08:20:44.132000+00:00
980,27086,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126402,Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIOYE126402,2023-12-13,2023-12-14T08:20:44.132000+00:00
981,27087,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126404,Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIOYE126404,2023-12-13,2023-12-14T08:20:44.132000+00:00
982,27088,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE126405,Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIOYE126405,2023-12-13,2023-12-14T08:20:44.132000+00:00
983,27089,info@ionity.eu,Chartres Gasville,"[1.580008, 48.468782]","Chartres-Gasville A11, 28300 Gasville-Oisème",FRIOYE12645,Chartres Gasville,Station dédiée à la recharge rapide,5,Accès libre,24/7,false,FRIOYE12645,2023-12-13,2023-12-14T08:20:44.132000+00:00


In [48]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[4.732409, 44.194832]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
977,27052,info@ionity.eu,IONITY GMBH,"[4.732409, 44.194832]","Aire de Mornas Village , A7, 84550 Mornas",FRIONE440300,IONITY Mornas Village,Station dédiée à la recharge rapide,16,Accès libre,24/7,FALSE,FRIONE4403,2023-03-21,2023-12-14T08:19:38.261000+00:00
1239,27751,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440301,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440301,2023-12-13,2023-12-14T08:20:44.132000+00:00
1240,27752,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440302,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440302,2023-12-13,2023-12-14T08:20:44.132000+00:00
1241,27753,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440303,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440303,2023-12-13,2023-12-14T08:20:44.132000+00:00
1242,27754,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440304,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440304,2023-12-13,2023-12-14T08:20:44.132000+00:00
1243,27755,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440305,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440305,2023-12-13,2023-12-14T08:20:44.132000+00:00
1244,27756,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440306,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440306,2023-12-13,2023-12-14T08:20:44.132000+00:00
1245,27757,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440307,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440307,2023-12-13,2023-12-14T08:20:44.132000+00:00
1246,27758,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440308,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440308,2023-12-13,2023-12-14T08:20:44.132000+00:00
1247,27759,info@ionity.eu,Mornas Village,"[4.732409, 44.194832]","Aire de Mornas Village, A7, 84550 Mornas",FRIOYE440309,Mornas Village,Station dédiée à la recharge rapide,18,Accès libre,24/7,false,FRIOYE440309,2023-12-13,2023-12-14T08:20:44.132000+00:00


In [27]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
18,119,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","400 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:00,false,FR3R3E10000849681,2023-11-22,2023-12-14T08:21:48.506000+00:00
19,120,exploitation@r3-charge.fr,R3,"[3.472424, 50.379911]","401 Rue Michel Chasles, 59494 Petite-Forêt",FR3R3P89616641,R3 - Petite Forêt,Parking privé à usage public,5,Accès libre,Mo-Su 07:00-22:01,false,FR3R3E10000849682,2023-11-22,2023-12-14T08:21:48.506000+00:00
20,121,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",708 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849691,2023-11-22,2023-12-14T08:21:48.506000+00:00
21,122,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",709 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849692,2023-11-22,2023-12-14T08:21:48.506000+00:00
22,137,exploitation@r3-charge.fr,R3,"[1.613217, 50.485562]",713 Av. François Godin Bis 62780 Cucq,FR3R3P89629589,R3 - Cucq,Parking privé à usage public,7,Accès libre,24/7,false,FR3R3E10000849851,2023-11-22,2023-12-14T08:21:48.506000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3298,72740,advenir@zeborne.com,GARAGE NELLO CHELLI - DIJON,"[5.065799, 47.354481]",23 RUE DES ARDENNES 21000 Dijon,FRZTLE22AC55087,Toyota - Dijon - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 07:45-12:00,Mo-Fr 13:45-19:00,Sat 09:00-...",false,FRZTLE22AC55087,2021-02-19,2023-12-14T08:22:46.201000+00:00
3299,72749,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2023-12-14T08:22:45.715000+00:00
3300,72750,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2023-12-14T08:22:45.715000+00:00
3301,72751,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2023-12-14T08:22:45.715000+00:00


In [50]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[5.2592028, 46.1746523]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3299,72749,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2023-12-14T08:22:45.715000+00:00
3300,72750,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2023-12-14T08:22:45.715000+00:00
3301,72751,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2023-12-14T08:22:45.715000+00:00
3302,72752,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2023-12-14T08:22:45.715000+00:00


In [51]:
itinerance_4.loc[itinerance_4.adresse_station == 'RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3299,72749,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57332,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57332,2021-04-08,2023-12-14T08:22:45.715000+00:00
3301,72751,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57334,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57334,2021-04-08,2023-12-14T08:22:45.715000+00:00


In [58]:
itinerance_4.loc[itinerance_4.adresse_station == ' RN 75 ZI DE NOIREFONTAINE 01250 Montagnat', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
3300,72750,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57333,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57333,2021-04-08,2023-12-14T08:22:45.715000+00:00
3302,72752,advenir@zeborne.com,TOYOTA MONTAGNAT - DJB BOURG,"[5.2592028, 46.1746523]",RN 75 ZI DE NOIREFONTAINE 01250 Montagnat,FRZTLE22AC57335,Toyota - Montagnat - 22kW AC,Parking public,1,Accès libre,"Mo-Fr 08:00–12:00,Mo-Fr 14:00–19:00,Sat 09:00-...",false,FRZTLE22AC57335,2021-04-08,2023-12-14T08:22:45.715000+00:00


- exemple : Stations Ouest charge (double saisie par deux opérateurs)

In [29]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-3.577855, 48.771084]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1462,47979,support@alizecharge.fr,Ouest Charge,"[-3.577855, 48.771084]",0 parking place du peillet 22560 TREBEURDEN,FRS22P22343001,Trebeurden-Place du Peillet,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS22E223430012,2030-04-12,2023-12-14T10:47:44.030000+00:00
1633,48182,sav@izivia.com,OUEST CHARGE - 22,"[-3.577855, 48.771084]",PARKING PLACE DU PEILLET,FRS22PSDE22311,TREBEURDEN-PLACE DU PEILLET,Voirie,2,Accès libre,24/7,false,FRS22ESDE22311,2023-04-07,2023-12-14T08:17:59.425000+00:00
1634,48183,sav@izivia.com,OUEST CHARGE - 22,"[-3.577855, 48.771084]",PARKING PLACE DU PEILLET,FRS22PSDE22312,TREBEURDEN-PLACE DU PEILLET,Voirie,2,Accès libre,24/7,false,FRS22ESDE22312,2023-04-07,2023-12-14T08:17:59.425000+00:00


### Cohérence station - coordonnées
- 323 pdc sont associés à des stations avec plusieurs coordonnées

In [30]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
742,20249,web@freshmile.com,Freshmile,"[4.636316, 44.26687]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1ERHZF2,2022-11-09,2023-12-14T08:23:11.304000+00:00
743,20418,web@freshmile.com,Freshmile,"[5.85536994, 43.11617585]",Boulevard de l'Europe 83500 La Seyne sur Mer,FRFR1PLLHUCH3KM1EX6J,"La Seyne sur Mer, Boulevard de l'Europe",Voirie,10,Accès libre,24/7,false,FRFR1ERUKN1,2023-12-01,2023-12-14T08:23:11.304000+00:00
744,20471,web@freshmile.com,Freshmile,"[7.61761, 48.5401]",Boulevard de l'Europe 83500 La Seyne sur Mer,FRFR1PLLHUCH3KM1EX6J,"La Seyne sur Mer, Boulevard de l'Europe",Voirie,10,Accès libre,24/7,false,FRFR1ERWVZ2,2023-12-01,2023-12-14T08:23:11.304000+00:00
750,20759,web@freshmile.com,Freshmile,"[2.8719224, 49.48156493]",Rue De Gand 59200 Tourcoing,FRFR1PKHNTCSOFD9,"Tourcoing, Rue de Gand",Voirie,14,Accès libre,24/7,false,FRFR1ESVMU1,2023-12-01,2023-12-14T08:23:11.304000+00:00
772,21252,web@freshmile.com,Freshmile,"[3.139213798, 45.89527515]",6 Rue Louis Armstrong 63200 Riom,FRFR1PE2YEK1N0HS,"Riom, Rue Louis Armstrong",Voirie,5,Accès libre,24/7,false,FRFR1EUMCR1,2023-11-30,2023-12-14T08:23:11.304000+00:00
775,21299,web@freshmile.com,Freshmile,"[-0.749697, 46.642619]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUQWM1,2023-11-30,2023-12-14T08:23:11.304000+00:00
776,21300,web@freshmile.com,Freshmile,"[-1.613876, 47.249316]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUQWM2,2023-11-30,2023-12-14T08:23:11.304000+00:00
777,21311,web@freshmile.com,Freshmile,"[-1.613876, 47.249316]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUSRM1,2023-11-30,2023-12-14T08:23:11.304000+00:00
778,21312,web@freshmile.com,Freshmile,"[-1.613876, 47.249316]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUSRM2,2023-11-30,2023-12-14T08:23:11.304000+00:00
796,21968,web@freshmile.com,Freshmile,"[4.7394, 45.97827]",733 Avenue Théodore Braun 69400 Villefranche-s...,FRFR1PRPDMF5LYE0,"Villefranche-sur-Saône, 733 Av. Théodore Braun",Voirie,13,Accès libre,24/7,false,FRFR1EWPDN2,2023-12-01,2023-12-14T08:23:11.304000+00:00


In [30]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
742,20249,web@freshmile.com,Freshmile,"[4.636316, 44.26687]",90 avenue d'Embrun 05000 Gap,FRFR1PEHNKRRPQNR,"Gap, 90 avenue d'Embrun - 3273",Voirie,6,Accès libre,24/7,false,FRFR1ERHZF2,2022-11-09,2023-12-14T08:23:11.304000+00:00
743,20418,web@freshmile.com,Freshmile,"[5.85536994, 43.11617585]",Boulevard de l'Europe 83500 La Seyne sur Mer,FRFR1PLLHUCH3KM1EX6J,"La Seyne sur Mer, Boulevard de l'Europe",Voirie,10,Accès libre,24/7,false,FRFR1ERUKN1,2023-12-01,2023-12-14T08:23:11.304000+00:00
744,20471,web@freshmile.com,Freshmile,"[7.61761, 48.5401]",Boulevard de l'Europe 83500 La Seyne sur Mer,FRFR1PLLHUCH3KM1EX6J,"La Seyne sur Mer, Boulevard de l'Europe",Voirie,10,Accès libre,24/7,false,FRFR1ERWVZ2,2023-12-01,2023-12-14T08:23:11.304000+00:00
750,20759,web@freshmile.com,Freshmile,"[2.8719224, 49.48156493]",Rue De Gand 59200 Tourcoing,FRFR1PKHNTCSOFD9,"Tourcoing, Rue de Gand",Voirie,14,Accès libre,24/7,false,FRFR1ESVMU1,2023-12-01,2023-12-14T08:23:11.304000+00:00
772,21252,web@freshmile.com,Freshmile,"[3.139213798, 45.89527515]",6 Rue Louis Armstrong 63200 Riom,FRFR1PE2YEK1N0HS,"Riom, Rue Louis Armstrong",Voirie,5,Accès libre,24/7,false,FRFR1EUMCR1,2023-11-30,2023-12-14T08:23:11.304000+00:00
775,21299,web@freshmile.com,Freshmile,"[-0.749697, 46.642619]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUQWM1,2023-11-30,2023-12-14T08:23:11.304000+00:00
776,21300,web@freshmile.com,Freshmile,"[-1.613876, 47.249316]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUQWM2,2023-11-30,2023-12-14T08:23:11.304000+00:00
777,21311,web@freshmile.com,Freshmile,"[-1.613876, 47.249316]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUSRM1,2023-11-30,2023-12-14T08:23:11.304000+00:00
778,21312,web@freshmile.com,Freshmile,"[-1.613876, 47.249316]",65 Avenue Du Général De Gaulle 85120 La Châtai...,FRFR1POL2TN2UB3M,"La Chataigneraie, 65 Avenue du General De Gaulle",Voirie,6,Accès libre,24/7,false,FRFR1EUSRM2,2023-11-30,2023-12-14T08:23:11.304000+00:00
796,21968,web@freshmile.com,Freshmile,"[4.7394, 45.97827]",733 Avenue Théodore Braun 69400 Villefranche-s...,FRFR1PRPDMF5LYE0,"Villefranche-sur-Saône, 733 Av. Théodore Braun",Voirie,13,Accès libre,24/7,false,FRFR1EWPDN2,2023-12-01,2023-12-14T08:23:11.304000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [31]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,6,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-12-14T08:17:35.133000+00:00
1,7,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-12-14T08:17:35.133000+00:00
2,8,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2023-12-14T08:17:35.133000+00:00
3,9,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-12-14T08:17:35.133000+00:00
5,11,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-12-14T08:17:35.133000+00:00
6,12,acelec@acelec-france.com,Camaïeu France,"[3.207047,50.685013]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EMZLV5283,2022-07-21,2023-12-14T08:17:35.133000+00:00
7,13,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-12-14T08:17:35.133000+00:00
8,14,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP7QU5233,2022-07-21,2023-12-14T08:17:35.133000+00:00
9,15,acelec@acelec-france.com,Camaïeu France,"[3.206962,50.685049]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EP9LD51287,2022-07-21,2023-12-14T08:17:35.133000+00:00
10,16,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-12-14T08:17:35.133000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [32]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
52,328,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2023-12-14T08:19:26.849000+00:00
53,329,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2023-12-14T08:19:26.849000+00:00
54,330,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2023-12-14T08:19:26.849000+00:00
55,331,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-12-14T08:19:26.849000+00:00
56,332,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2023-12-14T08:19:26.849000+00:00
57,333,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2023-12-14T08:19:26.849000+00:00


- exemple : plusieurs stations d'opérateurs différents avec les mêmes coordonnées -> suppression non effectuée lors d'un changement d'opérateur ?

In [33]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.658987, 48.542091]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2028,49433,support@alizecharge.fr,Ouest Charge,"[-4.658987, 48.542091]",Rue Auguste Caroff 29830 PLOUDALMEZEAU,FRS29P29178001,Ploudalmezeau-Rue Auguste Caroff,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E291780012,2049-04-12,2023-12-14T10:47:44.030000+00:00
2165,49686,sav@izivia.com,OUEST CHARGE - 29,"[-4.658987, 48.542091]",RUE AUGUSTE CAROFF,FRS29PSD2912011,PLOUDALMEZEAU-RUE AUGUSTE CAROFF,Voirie,2,Accès libre,24/7,false,FRS29ESD2912011,2023-04-14,2023-12-14T08:17:59.796000+00:00
2166,49687,sav@izivia.com,OUEST CHARGE - 29,"[-4.658987, 48.542091]",RUE AUGUSTE CAROFF,FRS29PSD2912012,PLOUDALMEZEAU-RUE AUGUSTE CAROFF,Voirie,2,Accès libre,24/7,false,FRS29ESD2912012,2023-04-14,2023-12-14T08:17:59.796000+00:00


In [34]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[-4.374469, 48.471878]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1978,49317,support@alizecharge.fr,Ouest Charge,"[-4.374469, 48.471878]",Route de Plabennec 29860 KERSAINT PLABENNEC,FRS29P29095001,Kersaint Plabennec-Route de Plabennec,Voirie,2,Accès libre,Mo-Su 00:00-23:59,FALSE,FRS29E290950012,2052-05-29,2023-12-14T10:47:44.030000+00:00
2459,49982,sav@izivia.com,OUEST CHARGE - 29,"[-4.374469, 48.471878]",ROUTE DE PLABENNEC,FRS29PSD296211,KERSAINT PLABENNEC-ROUTE DE PLABENNEC,Voirie,2,Accès libre,24/7,false,FRS29ESD296211,2023-04-14,2023-12-14T08:17:59.796000+00:00
2460,49983,sav@izivia.com,OUEST CHARGE - 29,"[-4.374469, 48.471878]",ROUTE DE PLABENNEC,FRS29PSD296212,KERSAINT PLABENNEC-ROUTE DE PLABENNEC,Voirie,2,Accès libre,24/7,false,FRS29ESD296212,2023-04-14,2023-12-14T08:17:59.796000+00:00


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [35]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
2435,56509,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 1,FRS64PMB649711,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649711,2023-12-13,2023-12-14T08:17:58.402000+00:00
2436,56510,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 1,FRS64PMB649712,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649712,2023-12-13,2023-12-14T08:17:58.402000+00:00
2437,56511,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 2,FRS64PMB649811,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649811,2023-12-13,2023-12-14T08:17:58.402000+00:00
2438,56512,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",0 PARKING ZENITH 2,FRS64PMB649812,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649812,2023-12-13,2023-12-14T08:17:58.402000+00:00


### Incoherence nbre_pdc
- 583 pdc ont un champ 'nbre_pdc' mal documenté

In [36]:
itinerance_4.loc[~itinerance_4['nbre_pdc - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
465,14651,assistance-commerciale@e-vadea.fr,e-Vadea,"[5.171659, 47.421321]",A31 - Aire de Dijon Brognon 21490 BROGNON (21),FREVAP21111A,e-Vadea - Brognon - A31 Dijon Brognon,Voirie,1,Accès libre,24/7,false,FREVAE21111AB4T2,2023-12-01,2023-12-14T08:22:57.941000+00:00
466,14652,assistance-commerciale@e-vadea.fr,e-Vadea,"[5.171659, 47.421321]",A31 - Aire de Dijon Brognon 21490 BROGNON (21),FREVAP21111A,e-Vadea - Brognon - A31 Dijon Brognon,Voirie,1,Accès libre,24/7,false,FREVAE21111AB5CCS,2023-12-01,2023-12-14T08:22:57.941000+00:00
467,14653,assistance-commerciale@e-vadea.fr,e-Vadea,"[5.171659, 47.421321]",A31 - Aire de Dijon Brognon 21490 BROGNON (21),FREVAP21111A,e-Vadea - Brognon - A31 Dijon Brognon,Voirie,1,Accès libre,24/7,false,FREVAE21111AB5CHADEMO,2023-12-01,2023-12-14T08:22:57.941000+00:00
468,14654,assistance-commerciale@e-vadea.fr,e-Vadea,"[5.171659, 47.421321]",A31 - Aire de Dijon Brognon 21490 BROGNON (21),FREVAP21111A,e-Vadea - Brognon - A31 Dijon Brognon,Voirie,1,Accès libre,24/7,false,FREVAE21111AB5T2,2023-12-01,2023-12-14T08:22:57.941000+00:00
469,14655,assistance-commerciale@e-vadea.fr,e-Vadea,"[5.171659, 47.421321]",A31 - Aire de Dijon Brognon 21490 BROGNON (21),FREVAP21111A,e-Vadea - Brognon - A31 Dijon Brognon,Voirie,1,Accès libre,24/7,false,FREVAE21111AB6CCS,2023-12-01,2023-12-14T08:22:57.941000+00:00
470,14656,assistance-commerciale@e-vadea.fr,e-Vadea,"[5.171659, 47.421321]",A31 - Aire de Dijon Brognon 21490 BROGNON (21),FREVAP21111A,e-Vadea - Brognon - A31 Dijon Brognon,Voirie,1,Accès libre,24/7,false,FREVAE21111AB6CHADEMO,2023-12-01,2023-12-14T08:22:57.941000+00:00
471,14657,assistance-commerciale@e-vadea.fr,e-Vadea,"[5.171659, 47.421321]",A31 - Aire de Dijon Brognon 21490 BROGNON (21),FREVAP21111A,e-Vadea - Brognon - A31 Dijon Brognon,Voirie,1,Accès libre,24/7,false,FREVAE21111AB6T2,2023-12-01,2023-12-14T08:22:57.941000+00:00
472,14658,assistance-commerciale@e-vadea.fr,e-Vadea,"[5.1724724, 47.4218283]",A31 - Aire de Dijon Spoy 21490 BROGNON (21),FREVAP21111B,e-Vadea - Brognon - A31 Dijon Spoy,Voirie,1,Accès libre,24/7,false,FREVAE21111BB3CCS,2023-12-01,2023-12-14T08:22:57.941000+00:00
473,14659,assistance-commerciale@e-vadea.fr,e-Vadea,"[5.1724724, 47.4218283]",A31 - Aire de Dijon Spoy 21490 BROGNON (21),FREVAP21111B,e-Vadea - Brognon - A31 Dijon Spoy,Voirie,1,Accès libre,24/7,false,FREVAE21111BB3CHADEMO,2023-12-01,2023-12-14T08:22:57.941000+00:00
474,14660,assistance-commerciale@e-vadea.fr,e-Vadea,"[5.1724724, 47.4218283]",A31 - Aire de Dijon Spoy 21490 BROGNON (21),FREVAP21111B,e-Vadea - Brognon - A31 Dijon Spoy,Voirie,1,Accès libre,24/7,false,FREVAE21111BB3T2,2023-12-01,2023-12-14T08:22:57.941000+00:00


### coherence station - enseigne
- 5 stations ont plusieurs noms d'enseigne

In [41]:
itinerance_4.loc[~itinerance_4['nom_enseigne - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
699,16613,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2023-12-14T08:21:08.880000+00:00
700,16614,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2023-12-14T08:21:08.880000+00:00
701,16615,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2023-12-14T08:21:08.880000+00:00
1309,35269,contact@rirodo.fr,LAST MILES SOLUTIONS,"[5.39, 43.27]",20 Rue Negresko,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRLMSE1897228,2023-09-06,2023-12-14T08:25:38.396000+00:00
1383,44674,contact@rirodo.fr,MONTA,"[5.39, 43.27]",20 Rue NEGRESKO,FRRIRE2804856,PARKING NEGRESKO,Parking privé à usage public,2,Accès libre,24/7,false,FRRIRE2804856,2023-09-06,2023-12-14T08:25:38.396000+00:00


In [42]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRFR1EAYRU', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
15339,15916,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2023-12-14T08:23:29.501000+00:00
15341,15918,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2023-12-14T08:23:29.501000+00:00
15343,15920,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU3,2022-08-25,2023-12-14T08:23:29.501000+00:00
16036,16613,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2023-12-14T08:21:08.880000+00:00
16037,16614,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2023-12-14T08:21:08.880000+00:00
16038,16615,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS3,2022-12-09,2023-12-14T08:21:08.880000+00:00


In [43]:
itinerance_init.loc[itinerance_init.nom_enseigne == 'RME Loos (FR-FR1)', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
15339,15916,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel jean colonna d'ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU1,2022-12-25,2023-12-14T08:23:29.501000+00:00
15341,15918,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.02,50.61]",rue du colonnel ornano loos,FRFR1EAYRU,"Loos, Parking de la Gare",Parking public,3,Accès libre,24/7,false,FRFR1EAYRU2,2022-08-25,2023-12-14T08:23:29.501000+00:00
16036,16613,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS1,2022-12-09,2023-12-14T08:21:08.880000+00:00
16037,16614,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.014273, 50.615449]",12 Rue Salengro LOOS,FRFR1EAYRU,"Loos, Parking Salengro",Parking public,3,Accès libre,24/7,FALSE,FRFR1EDXS2,2022-12-09,2023-12-14T08:21:08.880000+00:00
18995,19572,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS1,2022-10-04,2023-12-14T08:23:11.072000+00:00
18997,19574,contact@electricite-loos.fr,RME Loos (FR-FR1),"[3.009613, 50.617509]",12 Rue Salengro LOOS,FRFR1EPJXS,"Loos, Parking CIL",Parking public,3,Accès libre,24/7,FALSE,FRFR1EPJXS2,2022-10-04,2023-12-14T08:23:11.072000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire (15)

In [39]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues (583)

In [40]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1716', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
68937,70581,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17161,2023-07-06,2023-12-14T08:18:52.971000+00:00
68938,70582,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,TRUE,FRV75EPX17162,2023-07-06,2023-12-14T08:18:52.971000+00:00
68939,70583,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17163,2023-07-06,2023-12-14T08:18:52.971000+00:00
68940,70584,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17164,2023-07-06,2023-12-14T08:18:52.971000+00:00
68941,70585,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17165,2023-07-06,2023-12-14T08:18:52.971000+00:00
68942,70586,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17166,2023-07-06,2023-12-14T08:18:52.971000+00:00
68943,70587,supervision-ev.france@totalenergies.com,Belib',"[2.3242009, 48.8957574]",47 Rue Navier 75017 Paris,FRV75PPX1716,Paris | Rue Navier 47,Voirie,7,Accès libre,24/7,FALSE,FRV75EPX17167,2023-07-06,2023-12-14T08:18:52.971000+00:00
